In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
df = pd.read_csv("cleaned_train.csv")
df.head(2)

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Windows,Online Medical Journal,Website,www.cancertherapyadvisor.com,"['general', 'endocrine', 'false', 'medicine', ...",NaN,0.0
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Macintosh,Online Medical Journal,Website,www.cancertherapyadvisor.com,"['bone marrow', 'radiography', 'chronic', 'onc...",NaN,0.0


In [4]:
def method(x):
    s = ""
    l = []
    for i in x:
        i = i.split(" ")
        l.append("".join(i))
    return " ".join(l)
df['KEYWORDS'] = df['KEYWORDS'].apply(lambda x: eval(x))
df["KEYWORDS"] = df["KEYWORDS"].apply(method)

### Trying relating ```KEYWORDS``` and ```IS_HCP```

In [5]:
# from sklearn.model_selection import train_test_split

# X = df["KEYWORDS"]
# y = df["IS_HCP"]
# train_ratio = 0.7
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_ratio, stratify=y)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer()
# vectorizer.fit(X_train)

In [7]:
# vectorizer.get_feature_names_out()

In [8]:
# X_train_vec =  vectorizer.transform(X_train)

In [9]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score
# clf = LogisticRegression()
# clf.fit(X_train_vec, y_train)
# k_test = vectorizer.transform(X_test)
# y_pred = clf.predict(k_test)
# accuracy_score(y_test, y_pred)

In [10]:
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier()
# clf.fit(X_train_vec, y_train)
# k_test = vectorizer.transform(X_test)
# y_pred = clf.predict(k_test)
# accuracy_score(y_test, y_pred)

In [11]:
# from sklearn.svm import SVC
# clf = SVC()
# clf.fit(X_train_vec, y_train)
# k_test = vectorizer.transform(X_test)
# y_pred = clf.predict(k_test)
# accuracy_score(y_test, y_pred)


In [12]:
# from sklearn.ensemble import AdaBoostClassifier
# clf = AdaBoostClassifier()
# clf.fit(X_train_vec, y_train)
# k_test = vectorizer.transform(X_test)
# y_pred = clf.predict(k_test)
# accuracy_score(y_test, y_pred)


### On experimenting, there is a very huge correlation between ```IS_HCP``` and ```KEYWORDS```.

In [13]:
data = pd.DataFrame(df[["DEVICETYPE","PLATFORMTYPE","URL","KEYWORDS","IS_HCP"]])
data.head(2)

,DEVICETYPE,PLATFORMTYPE,URL,KEYWORDS,IS_HCP
0,Desktop,Online Medical Journal,www.cancertherapyadvisor.com,general endocrine false medicine surgery urolo...,0.0
1,Desktop,Online Medical Journal,www.cancertherapyadvisor.com,bonemarrow radiography chronic oncology psychi...,0.0


In [14]:
counter = dict(data.URL.value_counts())

In [15]:
for i in range(len(data)):
    if counter[data["URL"].iloc[i]]<5:
        data["URL"].iloc[i] = "others"

C:\Users\athar\AppData\Local\Temp\ipykernel_15460\1860523055.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["URL"].iloc[i] = "others"
C:\Users\athar\AppData\Local\Temp\ipykernel_15460\1860523055.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["URL"].iloc[i] = "others"
C:\Users\athar\AppData\Local\Temp\ipykernel_15460\1860523055.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["URL"].iloc[i] = "others"
C:\User

In [16]:
data.head(2)

,DEVICETYPE,PLATFORMTYPE,URL,KEYWORDS,IS_HCP
0,Desktop,Online Medical Journal,www.cancertherapyadvisor.com,general endocrine false medicine surgery urolo...,0.0
1,Desktop,Online Medical Journal,www.cancertherapyadvisor.com,bonemarrow radiography chronic oncology psychi...,0.0


In [61]:
new1 = pd.get_dummies(data, columns=["URL"],drop_first=True)

# do label encoding for url
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data["URL"] = le.fit_transform(data["URL"])
# also do one hot encoding for url use sklearn


In [18]:
vectorizer = CountVectorizer()
vectorizer.fit(data["KEYWORDS"])

CountVectorizer()

In [19]:
vectorizer.get_feature_names_out()

array(['19', 'ablationtechniques', 'abortion', ..., 'zoster',
       'zostersineherpete', 'zostervirus'], dtype=object)

In [20]:
kw_vector =  vectorizer.transform(data["KEYWORDS"])
new2 = pd.DataFrame(kw_vector.toarray())

In [37]:
DATA = pd.concat([new1,new2],axis=1)

In [40]:
DATA["IS_HCP"] = data["IS_HCP"]

In [41]:
DATA.select_dtypes(include='object').columns

Index([], dtype='object')

In [43]:
from sklearn.model_selection import train_test_split
X = DATA.drop(["IS_HCP"],axis=1)
X.columns = X.columns.astype(str)
y = DATA["IS_HCP"]
train_ratio = 0.7
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_ratio, stratify=y)

In [45]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score
# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# accuracy_score(y_test, y_pred)

In [44]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9756435643564356

In [46]:
# from sklearn.svm import SVC
# clf = SVC()
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# accuracy_score(y_test, y_pred)


In [47]:
# from sklearn.ensemble import AdaBoostClassifier
# clf = AdaBoostClassifier()
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# accuracy_score(y_test, y_pred)


In [48]:
test = pd.read_csv("cleaned_test.csv")

In [49]:
kw_vector =  vectorizer.transform(test["KEYWORDS"])
new = pd.DataFrame(kw_vector.toarray())

In [50]:
y_pred = clf.predict(new)

c:\Users\athar\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [53]:
# convert it to int

y_pred = y_pred.astype(int)

In [55]:
y_pred.shape, test.shape

((28493, 1), (28493, 12))

In [56]:
test["IS_HCP"] = y_pred

In [58]:
sub = test[["ID","IS_HCP"]]
sub.to_csv("sub.csv",index=False)